In [1]:
!pip install pulp

import pandas as pd
from pulp import *

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 14.2 MB 14.7 MB/s 


In [233]:
import pandas as pd

In [260]:
# distance_matrix = pd.read_excel('./광주광역시_경계구역_읍면동_충장동통합.xlsx', usecols = 'B : GM')
distance_matrix = pd.read_excel('./신창동_p-median_거리행렬.xlsx')
distance_matrix

,ID,광주 광산구 왕버들로251번길 27,광주 광산구 왕버들로252번길 46,광주 광산구 수등로 287
0,광주 광산구 왕버들로251번길 27,0.000000,436.318289,1321.315716
1,광주 광산구 왕버들로252번길 46,436.318289,0.000000,1047.535542
2,광주 광산구 수등로 287,1321.315716,1047.535542,0.000000


In [261]:
## 인구수 가중치로 사용x
# pop = pd.read_excel('./행정안전부_법정동_주민등록_인구수.xlsx')

In [262]:
# pop_final = list(pop.iloc[:, 1]) # 인구수
final = distance_matrix[:] # 법정동 거리행렬

In [263]:
final = final.set_index('ID')
final

,광주 광산구 왕버들로251번길 27,광주 광산구 왕버들로252번길 46,광주 광산구 수등로 287
ID,,,
광주 광산구 왕버들로251번길 27,0.000000,436.318289,1321.315716
광주 광산구 왕버들로252번길 46,436.318289,0.000000,1047.535542
광주 광산구 수등로 287,1321.315716,1047.535542,0.000000


In [264]:
region_name = final.iloc[:1,:] # 법정동명
region_name_final = list(region_name) # 법정동명 리스트로 저장
location = region_name_final # 동 이름 전처리

In [265]:
location

['광주 광산구 왕버들로251번길 27', '광주 광산구 왕버들로252번길 46', '광주 광산구 수등로 287']

In [266]:
# a = dict(zip(location, pop_final))
# 위치 개수
len(location)

3

In [267]:
## 거리행렬 정의
D = {}
for i in range(len(location)): # 딕셔너리 형태로 한 동당 거리행렬 정의
    D[i] = dict(zip(location[i:], [dict(zip(location, list(final.iloc[i,:])))]))

merged = {}
for i in range(len(location)):
    merged = {**merged, **D[i]} # 193 + 193 거리행렬 정의    

In [268]:
# 거리행렬 확인
merged 

{'광주 광산구 수등로 287': {'광주 광산구 수등로 287': 0.0,
  '광주 광산구 왕버들로251번길 27': 1321.315716477261,
  '광주 광산구 왕버들로252번길 46': 1047.535542138273},
 '광주 광산구 왕버들로251번길 27': {'광주 광산구 수등로 287': 1321.315716476755,
  '광주 광산구 왕버들로251번길 27': 0.0,
  '광주 광산구 왕버들로252번길 46': 436.3182887397199},
 '광주 광산구 왕버들로252번길 46': {'광주 광산구 수등로 287': 1047.535542137904,
  '광주 광산구 왕버들로251번길 27': 436.3182887397905,
  '광주 광산구 왕버들로252번길 46': 0.0}}

In [269]:
p = 1 # 공중전화부스 선택 개수

In [270]:
# P-Median 모델을 통해 최적지 선정하는 코드
# X -> 변수명 설정
X = LpVariable.dicts('%s_%s', (location, location), cat = 'Binary',
                     lowBound = 0,
                     upBound = 1)

In [271]:
X

{'광주 광산구 수등로 287': {'광주 광산구 수등로 287': 광주_광산구_수등로_287_광주_광산구_수등로_287,
  '광주 광산구 왕버들로251번길 27': 광주_광산구_수등로_287_광주_광산구_왕버들로251번길_27,
  '광주 광산구 왕버들로252번길 46': 광주_광산구_수등로_287_광주_광산구_왕버들로252번길_46},
 '광주 광산구 왕버들로251번길 27': {'광주 광산구 수등로 287': 광주_광산구_왕버들로251번길_27_광주_광산구_수등로_287,
  '광주 광산구 왕버들로251번길 27': 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로251번길_27,
  '광주 광산구 왕버들로252번길 46': 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로252번길_46},
 '광주 광산구 왕버들로252번길 46': {'광주 광산구 수등로 287': 광주_광산구_왕버들로252번길_46_광주_광산구_수등로_287,
  '광주 광산구 왕버들로251번길 27': 광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로251번길_27,
  '광주 광산구 왕버들로252번길 46': 광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로252번길_46}}

 LpMaximize: 최대화 문제 -> 3개중 2개 추출할 경우(ex.신창동) 사용

 LpMinimize: 최소화 문제 

In [279]:
# prob = LpProblem('P Median', LpMaximize) # 목적 함수
prob = LpProblem('P Median', LpMinimize)

/usr/local/lib/python3.7/dist-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [280]:
# prob += sum(sum(a[i]*merged[i][j] * X[i][j] for j in location)for i in location)
prob += sum(sum(merged[i][j] * X[i][j] for j in location)for i in location) # a[i] 값 제거(인구가중치 제거)
prob += sum(X[i][i] for i in location) == p # p=193

In [281]:
for i in location: prob += sum(X[i][j] for j in location) == 1 # 제약식

for i in location:
    for j in location: prob += X[i][j] <= X[j][j]

In [282]:
prob.writeLP("p-median_Gwangju_Whole.lp")
prob.solve() # prob 문제 풀렸으면 1 출력

1

In [283]:
# prob
# 목적함수, 제약식 출력

pulp.value(prob.objective)

prob

P_Median:
MINIMIZE
1321.315716477261*광주_광산구_수등로_287_광주_광산구_왕버들로251번길_27 + 1047.535542138273*광주_광산구_수등로_287_광주_광산구_왕버들로252번길_46 + 1321.315716476755*광주_광산구_왕버들로251번길_27_광주_광산구_수등로_287 + 436.3182887397199*광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로252번길_46 + 1047.535542137904*광주_광산구_왕버들로252번길_46_광주_광산구_수등로_287 + 436.3182887397905*광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로251번길_27 + 0.0
SUBJECT TO
_C1: 광주_광산구_수등로_287_광주_광산구_수등로_287 + 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로251번길_27
 + 광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로252번길_46 = 1

_C2: 광주_광산구_왕버들로251번길_27_광주_광산구_수등로_287
 + 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로251번길_27
 + 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로252번길_46 = 1

_C3: 광주_광산구_왕버들로252번길_46_광주_광산구_수등로_287
 + 광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로251번길_27
 + 광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로252번길_46 = 1

_C4: 광주_광산구_수등로_287_광주_광산구_수등로_287 + 광주_광산구_수등로_287_광주_광산구_왕버들로251번길_27
 + 광주_광산구_수등로_287_광주_광산구_왕버들로252번길_46 = 1

_C5: 0 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로251번길_27 <= 0

_C6: 광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로252번길_46
 - 광주_광산구_왕버들로252번길_46_광주_

In [284]:
# 위치 확인
location

['광주 광산구 왕버들로251번길 27', '광주 광산구 왕버들로252번길 46', '광주 광산구 수등로 287']

In [285]:
# 1.0 값이 나오는 동 추출
# varValue -> 최적해 출력
for v in prob.variables():
  print(v.name + "=", v.varValue)

광주_광산구_수등로_287_광주_광산구_수등로_287= 0.0
광주_광산구_수등로_287_광주_광산구_왕버들로251번길_27= 0.0
광주_광산구_수등로_287_광주_광산구_왕버들로252번길_46= 1.0
광주_광산구_왕버들로251번길_27_광주_광산구_수등로_287= 0.0
광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로251번길_27= 0.0
광주_광산구_왕버들로251번길_27_광주_광산구_왕버들로252번길_46= 1.0
광주_광산구_왕버들로252번길_46_광주_광산구_수등로_287= 0.0
광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로251번길_27= 0.0
광주_광산구_왕버들로252번길_46_광주_광산구_왕버들로252번길_46= 1.0
